In [142]:
import glob
import os
import tgt
import statistics
import pandas as pd
import numpy as np
from collections import defaultdict

In [193]:
# read gold files
gold_dir = 'data/gold_utts'
gold_dict = {}
for phone_file in glob.glob(f'{gold_dir}/**/*.TextGrid', recursive=True):
    utt_id = os.path.basename(phone_file).split('.')[0]
    spkr_id = utt_id.split('-')[-1]
    
    # quickly remove this utt (contains erroneous "fs", supposed to be "fs_")
    # this utt is otherwise russian, not CS
    if utt_id == '0010_DoReCo_doreco_urum1249_UUM-TXT-AN-00000-A02':
        continue
    
    try:
        tgt_grid = tgt.io.read_textgrid(phone_file, encoding='utf-16')
    except:
        try:
            tgt_grid = tgt.io.read_textgrid(phone_file, encoding='utf-8')        
        except:
            print(f'** SKIP (encoding issues): {utt_id}')
            continue
            
    phone_tier = tgt_grid.get_tier_by_name(f'ph@{spkr_id}')
    utt_list = []
    for phone_item in phone_tier:
        phone = phone_item.text.replace('r\\_r', 'r')
        
        if phone.startswith('<'):  # <<fs>word> or <p:>
            continue
        
        start = round(phone_item.start_time, 3)
        end = round(phone_item.end_time, 3)
        utt_list.append((phone, start, end))
    gold_dict[utt_id] = utt_list

In [186]:
gold_dict

{'0006_DoReCo_doreco_urum1249_UUM-TXT-VL-00000-B08': [('o', 0.0, 0.098),
  ('c', 0.098, 0.21),
  ('9', 0.21, 0.348),
  ('v', 0.348, 0.378),
  ('y', 0.378, 0.418),
  ('m', 0.418, 0.448),
  ('y', 0.448, 0.5),
  ('z', 0.5, 0.588),
  ('d', 0.588, 0.623),
  ('{', 0.623, 0.716),
  ('v', 0.716, 0.784),
  ('a', 0.784, 0.886),
  ('4', 0.886, 0.906),
  ('M', 0.906, 0.957),
  ('d', 0.957, 1.056),
  ('i', 1.056, 1.1),
  ('J\\', 1.1, 1.18),
  ('y', 1.18, 1.22),
  ('z', 1.22, 1.299),
  ('{', 1.299, 1.424),
  ('l', 1.424, 1.53),
  ('p', 1.53, 1.61),
  ('a', 1.61, 1.78),
  ('4', 1.78, 1.814),
  ('k', 1.814, 1.942),
  ('v', 2.43, 2.46),
  ('a', 2.46, 2.57),
  ('4', 2.57, 2.6),
  ('M', 2.6, 2.67),
  ('d', 2.67, 2.758),
  ('i', 2.758, 2.814)],
 '0004_DoReCo_doreco_urum1249_UUM-TXT-FM-00000-A03': [('n', 0.0, 0.062),
  ('{', 0.062, 0.19),
  ('a', 0.19, 0.314),
  ('n', 0.314, 0.48),
  ('i', 0.921, 0.951),
  ('S', 0.951, 1.023),
  ('t', 1.023, 1.107),
  ('e', 1.107, 1.167),
  ('m', 1.167, 1.21),
  ('b', 1.21

In [3]:
test_urum_tsv = 'data/test_urum_utts2.tsv'
test_df = pd.read_csv(test_urum_tsv, sep='\t')

In [4]:
test_df

,utt_id,spkr_id,lang,text,dur,file_id
0,0001_DoReCo_doreco_urum1249_UUM-TXT-AN-00000-A01,A01,urum,bänä ğala gäldılär iki yuz el iräli trapezondadan,4.879,doreco_urum1249_UUM-TXT-AN-00000-A01
1,0001_DoReCo_doreco_urum1249_UUM-TXT-AN-00000-A02,A02,urum,dädäm dierdi ki biz gäldıh gürjistana gürjista...,8.130,doreco_urum1249_UUM-TXT-AN-00000-A02
2,0001_DoReCo_doreco_urum1249_UUM-TXT-AN-00000-B08,B08,urum,bizım halh gäldi turtsiadan onučun ki dad verm...,5.283,doreco_urum1249_UUM-TXT-AN-00000-B08
3,0001_DoReCo_doreco_urum1249_UUM-TXT-AN-00000-B13,B13,urum,näsıl bizım halh gäldi gürjüstanda,2.391,doreco_urum1249_UUM-TXT-AN-00000-B13
4,0001_DoReCo_doreco_urum1249_UUM-TXT-CL-00000-A03,A03,urum,ein ičın ğurban olmıšam sizä,2.124,doreco_urum1249_UUM-TXT-CL-00000-A03
...,...,...,...,...,...,...
129,0017_DoReCo_doreco_urum1249_UUM-TXT-FM-00000-A03,A03,urum,ambelä išlär,0.744,doreco_urum1249_UUM-TXT-FM-00000-A03
130,0017_DoReCo_doreco_urum1249_UUM-TXT-VL-00000-A01,A01,urum,äp düšünerdılär halh getmäh etmäh šindi o išlä...,2.666,doreco_urum1249_UUM-TXT-VL-00000-A01
131,0017_DoReCo_doreco_urum1249_UUM-TXT-VL-00000-A02,A02,urum,čoğ šei čäkierıh dağıldıh äp äpımız bir erä ge...,5.350,doreco_urum1249_UUM-TXT-VL-00000-A02
132,0018_DoReCo_doreco_urum1249_UUM-TXT-VL-00000-A01,A01,urum,äp düšünerlär burda yašamah,1.216,doreco_urum1249_UUM-TXT-VL-00000-A01


In [5]:
urum_test_utt_list = test_df['utt_id']
urum_test_utt_list

0      0001_DoReCo_doreco_urum1249_UUM-TXT-AN-00000-A01
1      0001_DoReCo_doreco_urum1249_UUM-TXT-AN-00000-A02
2      0001_DoReCo_doreco_urum1249_UUM-TXT-AN-00000-B08
3      0001_DoReCo_doreco_urum1249_UUM-TXT-AN-00000-B13
4      0001_DoReCo_doreco_urum1249_UUM-TXT-CL-00000-A03
                             ...                       
129    0017_DoReCo_doreco_urum1249_UUM-TXT-FM-00000-A03
130    0017_DoReCo_doreco_urum1249_UUM-TXT-VL-00000-A01
131    0017_DoReCo_doreco_urum1249_UUM-TXT-VL-00000-A02
132    0018_DoReCo_doreco_urum1249_UUM-TXT-VL-00000-A01
133    0018_DoReCo_doreco_urum1249_UUM-TXT-VL-00000-A02
Name: utt_id, Length: 134, dtype: object

In [110]:
test_all_df = pd.read_csv('data/test_all_utts.tsv', sep='\t')
test_all_df

,utt_id,spkr_id,lang,text,dur,file_id
0,0001_DoReCo_doreco_urum1249_UUM-TXT-AN-00000-A01,A01,urum,bänä ğala gäldılär iki yuz el iräli trapezondadan,4.879,doreco_urum1249_UUM-TXT-AN-00000-A01
1,0001_DoReCo_doreco_urum1249_UUM-TXT-AN-00000-A02,A02,urum,dädäm dierdi ki biz gäldıh gürjistana gürjista...,8.130,doreco_urum1249_UUM-TXT-AN-00000-A02
2,0001_DoReCo_doreco_urum1249_UUM-TXT-AN-00000-A03,A03,cs,birınji gäldılär bizım atadädä _tısyaču _vosem...,7.415,doreco_urum1249_UUM-TXT-AN-00000-A03
3,0001_DoReCo_doreco_urum1249_UUM-TXT-AN-00000-B08,B08,urum,bizım halh gäldi turtsiadan onučun ki dad verm...,5.283,doreco_urum1249_UUM-TXT-AN-00000-B08
4,0001_DoReCo_doreco_urum1249_UUM-TXT-AN-00000-B13,B13,urum,näsıl bizım halh gäldi gürjüstanda,2.391,doreco_urum1249_UUM-TXT-AN-00000-B13
...,...,...,...,...,...,...
271,0017_DoReCo_doreco_urum1249_UUM-TXT-VL-00000-A03,A03,cs,ordan čıhti _pervıe _sekretari _raikoma,3.890,doreco_urum1249_UUM-TXT-VL-00000-A03
272,0018_DoReCo_doreco_urum1249_UUM-TXT-VL-00000-A01,A01,urum,äp düšünerlär burda yašamah,1.216,doreco_urum1249_UUM-TXT-VL-00000-A01
273,0018_DoReCo_doreco_urum1249_UUM-TXT-VL-00000-A02,A02,urum,köti,0.395,doreco_urum1249_UUM-TXT-VL-00000-A02
274,0019_DoReCo_doreco_urum1249_UUM-TXT-VL-00000-A01,A01,cs,äp düšünerlär _pradalžat _zdes,2.956,doreco_urum1249_UUM-TXT-VL-00000-A01


In [111]:
russ_test_utt_list = test_all_df[test_all_df['lang'] == 'russ']['utt_id']
russ_test_utt_list

5      0001_DoReCo_doreco_urum1249_UUM-TXT-AN-00000-B16
26     0002_DoReCo_doreco_urum1249_UUM-TXT-AN-00000-A03
29     0002_DoReCo_doreco_urum1249_UUM-TXT-AN-00000-B16
51     0003_DoReCo_doreco_urum1249_UUM-TXT-AN-00000-A01
65     0003_DoReCo_doreco_urum1249_UUM-TXT-FE-00000-A03
68     0003_DoReCo_doreco_urum1249_UUM-TXT-FM-00000-A03
75     0004_DoReCo_doreco_urum1249_UUM-TXT-AN-00000-A01
84     0004_DoReCo_doreco_urum1249_UUM-TXT-CL-00000-A05
97     0004_DoReCo_doreco_urum1249_UUM-TXT-VL-00000-A03
102    0005_DoReCo_doreco_urum1249_UUM-TXT-AN-00000-A03
120    0005_DoReCo_doreco_urum1249_UUM-TXT-VL-00000-A03
128    0006_DoReCo_doreco_urum1249_UUM-TXT-AN-00000-B16
160    0008_DoReCo_doreco_urum1249_UUM-TXT-AN-00000-A02
181    0009_DoReCo_doreco_urum1249_UUM-TXT-AN-00000-A02
187    0009_DoReCo_doreco_urum1249_UUM-TXT-FE-00000-A02
201    0010_DoReCo_doreco_urum1249_UUM-TXT-AN-00000-B16
205    0010_DoReCo_doreco_urum1249_UUM-TXT-FE-00000-A02
220    0011_DoReCo_doreco_urum1249_UUM-TXT-FM-00

In [112]:
cs_test_utt_list = test_all_df[test_all_df['lang'] == 'cs']['utt_id']
cs_test_utt_list

2      0001_DoReCo_doreco_urum1249_UUM-TXT-AN-00000-A03
9      0001_DoReCo_doreco_urum1249_UUM-TXT-CL-00000-A07
11     0001_DoReCo_doreco_urum1249_UUM-TXT-FE-00000-A01
12     0001_DoReCo_doreco_urum1249_UUM-TXT-FE-00000-A02
13     0001_DoReCo_doreco_urum1249_UUM-TXT-FE-00000-A03
                             ...                       
266    0016_DoReCo_doreco_urum1249_UUM-TXT-VL-00000-A03
267    0017_DoReCo_doreco_urum1249_UUM-TXT-FM-00000-A02
271    0017_DoReCo_doreco_urum1249_UUM-TXT-VL-00000-A03
274    0019_DoReCo_doreco_urum1249_UUM-TXT-VL-00000-A01
275    0019_DoReCo_doreco_urum1249_UUM-TXT-VL-00000-A03
Name: utt_id, Length: 121, dtype: object

In [105]:
#test_all_df_indexed = test_all_df.set_index('utt_id', inplace=True)
#test_all_df_indexed
# test_all_df.loc['0001_DoReCo_doreco_urum1249_UUM-TXT-AN-00000-A02']['lang']

In [177]:
# read xsampa-ipa map
xsampa_map_file = 'data/map/cs-xsampa-to-ipa.map'
xsipa_dict = {}
ipa_to_xs = {}

with open(xsampa_map_file, 'r') as f:
    for line in f.readlines():
        items = line.strip().split('\t')
        if items:
            xsipa_dict[items[0]] = items[1]
            ipa_to_xs[items[1]] = items[0]

# xsipa_dict

In [178]:
len(ipa_to_xs)

43

In [179]:
len(xsipa_dict)

43

In [180]:
# read nat class file using pd
nat_class_file = 'data/map/natural_classes_cs.tsv'
df = pd.read_csv(nat_class_file, sep="\t", header=0)
classes = list(df.columns)
class_dict_nans = df.to_dict(orient='list')  # WARNING: contains some nans
# class_dict = {k: [x for x in v if ~np.isnan(x)] for k,v in class_dict_nans.items()}
class_dict_nans

{'vowel': ['œ', 'ɯ', 'a', 'e', 'i', 'o', 'u', 'y', 'æ', nan, 'ɨ', nan],
 'fricative': ['ɫ', 'ɣ', 'ʃ', 'ʒ', 'f', 's', 'sː', 'v', 'x', 'z', 'ʂ', 'ʐ'],
 'affricate': ['dʒ', 'tʃ', nan, nan, nan, nan, nan, nan, nan, nan, 'tɕ', nan],
 'nasal': ['m', 'mː', 'n', nan, nan, nan, nan, nan, nan, nan, nan, nan],
 'approx': ['ɫ', 'j', 'l', 'lː', nan, nan, nan, nan, nan, nan, nan, nan],
 'tap-trill': ['ɾ', 'r', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan],
 'stop': ['ɟ', 'b', 'c', 'd', 'dː', 'ɡ', 'k', 'p', 't', 'tː', nan, nan],
 'SIL': ['SIL', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]}

In [182]:
xs_to_class = {}  # ex. ['5']: 'vowel'
for nat_class, ipa_list in class_dict_nans.items():
    for ipa_item in ipa_list:
        if type(ipa_item) is str:  # not nan
            try:
                xs_to_class[ipa_to_xs[ipa_item]] = nat_class
            except:
                xs_to_class[ipa_item] = nat_class
xs_to_class

{'9': 'vowel',
 'M': 'vowel',
 'a': 'vowel',
 'e': 'vowel',
 'i': 'vowel',
 'o': 'vowel',
 'u': 'vowel',
 'y': 'vowel',
 '{': 'vowel',
 '1': 'vowel',
 '5': 'approx',
 'G': 'fricative',
 'S': 'fricative',
 'Z': 'fricative',
 'f': 'fricative',
 's': 'fricative',
 's:': 'fricative',
 'v': 'fricative',
 'x': 'fricative',
 'z': 'fricative',
 's`': 'fricative',
 'z`': 'fricative',
 'dZ': 'affricate',
 'tS': 'affricate',
 'ts\\': 'affricate',
 'm': 'nasal',
 'm:': 'nasal',
 'n': 'nasal',
 'j': 'approx',
 'l': 'approx',
 'l:': 'approx',
 '4': 'tap-trill',
 'r': 'tap-trill',
 'J\\': 'stop',
 'b': 'stop',
 'c': 'stop',
 'd': 'stop',
 'd:': 'stop',
 'g': 'stop',
 'k': 'stop',
 'p': 'stop',
 't': 'stop',
 't:': 'stop',
 'SIL': 'SIL'}

In [121]:
# read system predicted textgrids
# NOTE: works even if phone symbols are diff across gold and pred
#       i.e. don't need to map pred symbols back to orig using interlingual-MFA
def eval_tg(pred_dir: str, test_urum_only: bool, show_nat_class=False):
    pred_dict = {}
    bracket_word_count = 0

    for phone_file in glob.glob(f'{pred_dir}/**/*.TextGrid', recursive=True):
        utt_id = os.path.basename(phone_file).split('.')[0]
        if test_urum_only:
            if ~urum_test_utt_list.str.contains(utt_id).any():
                continue

        # exclude russ utts too
        if russ_test_utt_list.str.contains(utt_id).any():
            continue
            
        spkr_id = utt_id.split('-')[-1]

        try:
            tgt_grid = tgt.io.read_textgrid(phone_file, encoding='utf-16')
        except:
            try:
                tgt_grid = tgt.io.read_textgrid(phone_file, encoding='utf-8')        
            except:
                print(f'** SKIP (encoding issues): {utt_id}')
                continue

        # remove phones for which their words contain _ ex. "fs_elädır"
        word_tier = tgt_grid.get_tier_by_name('words')
        phone_tier = tgt_grid.get_tier_by_name('phones')
        for word in word_tier:
            if '_' in word.text:  # either foreign (starts with _) or other
                if not word.text.startswith('_'):
                    bracket_word_count += 1
                phone_tier.delete_annotations_between_timepoints(word.start_time, 
                                                                 word.end_time, 
                                                                 left_overlap=False, 
                                                                 right_overlap=False)

        utt_list = []
        for phone_item in phone_tier:
            phone = phone_item.text
            start = round(phone_item.start_time, 3)
            end = round(phone_item.end_time, 3)
            utt_list.append((phone, start, end))
        pred_dict[utt_id] = utt_list
    print(f'bracket_word_count: {bracket_word_count}')
    print(f'num pred utts: {len(pred_dict)}')
    
    # compare differences, pairwise
    diffs = []
    following_phones = []  # target phone (phone following onset)
    preceding_phones = []  # phone preceding the target. Could be SIL
    accuracies = 0  # +1 when pred midpoint lies within gold interval

    for file_id, pred_list in pred_dict.items():
        if file_id not in gold_dict:
            print(f'pred file not in gold: {file_id}')
            continue

        gold_list = gold_dict[file_id]

        # so far, they all match!
        if len(gold_list) != len(pred_list):
            print('*** mismatch ***', file_id, len(gold_list), len(pred_list))
            return

        # when gold and pred lists are same length
        for i_tup, gold_tup in enumerate(gold_list):
            pred_tup = pred_list[i_tup]
            # absolute diff of each phone's ONSET only
            # if want OFFSET, use tup[2]
            diff = abs(gold_tup[1] - pred_tup[1])
            diffs.append(diff)
            
            # accuracy
            pred_midpoint = (pred_tup[2] + pred_tup[1])/2
            if pred_midpoint > gold_tup[1] and pred_midpoint < gold_tup[2]:
                accuracies += 1
#             else:
#                 print(pred_tup, gold_tup)
#                 print(pred_midpoint)
            
            # update lists for following and preceding phones
            following_phones.append(gold_tup[0])
            if i_tup > 0:
                if gold_list[i_tup - 1][2] == gold_tup[1]:  # check if preceding phone and current phone are touching
                    preceding_phones.append(gold_list[i_tup - 1][0])
                    continue

            preceding_phones.append('SIL')
            
    print('Boundary Onset statistics:')
    print(f'# phones: {len(diffs)}')
    print(f'MEAN: {statistics.mean(diffs):0.3f}')
    print(f'MEDIAN: {statistics.median(diffs)}')
    print(f'MIN, MAX: {min(diffs)}, {max(diffs)}')

    within_window = 0
    window_sec = 0.02
    for diff_item in diffs:
        if diff_item < window_sec:
            within_window += 1

    print(f'WINDOW {window_sec}s (PRECISION) %): {float(within_window)*100/len(diffs):0.3f}')
    print(f'ACCURACY (test midpoint within gold interval) %: {float(accuracies)*100/len(diffs):0.3f}')
    
    # nat class
    if not show_nat_class:
        return
    diffs_np = np.array(diffs)
#     print(diffs_np)
#     return
    prec_np = np.array(preceding_phones)
    foll_np = np.array(following_phones)
    
    for phone_location, phone_list in {'PREC (offset)': prec_np, 'FOLL (onset)': foll_np}.items():
        print(f'\n{phone_location}\n')
        for class_name in classes:
#         for class_name in sorted(list(phone_set)):

            class_diffs = []
            within_window = 0
#             window_msec = 20  # hard-coded for now
            for phone_i, phone_xs in enumerate(phone_list):
                try:
                    phone = xsipa_dict[phone_xs]
                except:
                    phone = phone_xs
                if phone in class_dict_nans[class_name]:
#                 if phone == class_name:  # fine-grained
                    class_diffs.append(diffs_np[phone_i])
                    if diffs_np[phone_i] < window_sec:  # used to be window_msec
                        within_window += 1

            print(f'\n** {class_name} **')

            try:
                print(f'WINDOW ({window_sec}s) %: {float(within_window)*100/len(class_diffs):0.3f}\tCOUNT:{len(class_diffs)}')
                # print(f'{float(within_window)*100/len(class_diffs):0.2f}')
                print(f'MEAN: {statistics.mean(class_diffs):0.2f}')
                print(f'MEDIAN: {statistics.median(class_diffs)}')
                print(f'MIN, MAX: {min(class_diffs)}, {max(class_diffs)}')
            except:
                print('[none]')

In [196]:
# BATCH read system predicted textgrids
# NOTE: works even if phone symbols are diff across gold and pred
#       i.e. don't need to map pred symbols back to orig using interlingual-MFA
def batch_eval_tg(pred_tg_dir: str, pred_dirs: list, 
                  test_urum_only: bool, show_nat_class=False, write_df_dir=None):
    results_dict = defaultdict(dict)
    
    for pred_dir in pred_dirs:
        pred_dict = {}
        bracket_word_count = 0

        for phone_file in glob.glob(f'{pred_tg_dir}/{pred_dir}/**/*.TextGrid', recursive=True):
            utt_id = os.path.basename(phone_file).split('.')[0]
            if test_urum_only:
                if ~urum_test_utt_list.str.contains(utt_id).any():
                    continue

            # exclude russ utts too
            if russ_test_utt_list.str.contains(utt_id).any():
                continue

            spkr_id = utt_id.split('-')[-1]

            try:
                tgt_grid = tgt.io.read_textgrid(phone_file, encoding='utf-16')
            except:
                try:
                    tgt_grid = tgt.io.read_textgrid(phone_file, encoding='utf-8')        
                except:
#                     print(f'** SKIP (encoding issues): {utt_id}')
                    continue

            # remove phones for which their words contain _ ex. "fs_elädır"
            word_tier = tgt_grid.get_tier_by_name('words')
            phone_tier = tgt_grid.get_tier_by_name('phones')
            for word in word_tier:
                if '_' in word.text:  # either foreign (starts with _) or other
                    if not word.text.startswith('_'):
                        bracket_word_count += 1
                    phone_tier.delete_annotations_between_timepoints(word.start_time, 
                                                                     word.end_time, 
                                                                     left_overlap=False, 
                                                                     right_overlap=False)

            utt_list = []
            for phone_item in phone_tier:
                phone = phone_item.text
                start = round(phone_item.start_time, 3)
                end = round(phone_item.end_time, 3)
                utt_list.append((phone, start, end))
            pred_dict[utt_id] = utt_list
#         print(f'bracket_word_count: {bracket_word_count}')
        results_dict[pred_dir]['# pred utts'] = len(pred_dict)

        # compare differences, pairwise
        diffs = []
        following_phones = []  # target phone (phone following onset)
        preceding_phones = []  # phone preceding the target. Could be SIL
        foll_phon_classes = []  # target phone (phone following onset)
        prec_phon_classes = []  # phone preceding the target. Could be SIL
        accuracy_list = []
        accuracies = 0  # +1 when pred midpoint lies within gold interval
        utt_id_list = []

        for file_id, pred_list in pred_dict.items():
            if file_id not in gold_dict:
#                 print(f'pred file not in gold: {file_id}')
                continue

            gold_list = gold_dict[file_id]

            # so far, they all match!
            if len(gold_list) != len(pred_list):
                print('*** mismatch ***', file_id, len(gold_list), len(pred_list))
                return

            # when gold and pred lists are same length
            for i_tup, gold_tup in enumerate(gold_list):
                pred_tup = pred_list[i_tup]
                # absolute diff of each phone's ONSET only
                # if want OFFSET, use tup[2]
                diff = abs(gold_tup[1] - pred_tup[1])
                diffs.append(round(diff, 6))

                # accuracy
                pred_midpoint = (pred_tup[2] + pred_tup[1])/2
                if pred_midpoint > gold_tup[1] and pred_midpoint < gold_tup[2]:
                    accuracies += 1
                    accuracy_list.append(1)
                else:
                    accuracy_list.append(0)
    #                 print(pred_tup, gold_tup)
    #                 print(pred_midpoint)

                # update lists for following and preceding phones
                following_phones.append(gold_tup[0])
                try:
                    foll_phon_classes.append(xs_to_class[gold_tup[0]])
                except:
                    print(file_id)
                    
                if i_tup > 0:
                    if gold_list[i_tup - 1][2] == gold_tup[1]:  # check if preceding phone and current phone are touching
                        preceding_phones.append(gold_list[i_tup - 1][0])
                        prec_phon_classes.append(xs_to_class[gold_list[i_tup - 1][0]])
                        utt_id_list.append(file_id)
                        continue

                preceding_phones.append('SIL')
                prec_phon_classes.append(xs_to_class['SIL'])
                utt_id_list.append(file_id)

#         print(f'Boundary Onset statistics: len(diffs): {len(diffs)}')
        results_dict[pred_dir]['# phones'] = len(diffs)
        results_dict[pred_dir]['MEAN'] = round(statistics.mean(diffs), 3)
        results_dict[pred_dir]['MEDIAN'] = statistics.median(diffs)
        results_dict[pred_dir]['MIN, MAX'] = (round(min(diffs), 3), round(max(diffs), 3))

        within_window = 0
        window_sec = 0.02
        for diff_item in diffs:
            if diff_item < window_sec:
                within_window += 1

        results_dict[pred_dir]['PRECISION'] = round(float(within_window)*100/len(diffs), 3)
        results_dict[pred_dir]['ACCURACY'] = round(float(accuracies)*100/len(diffs), 3)
        
        if write_df_dir:
            if not os.path.exists(write_df_dir):
                os.makedirs(write_df_dir)

            out_df_file = os.path.join(write_df_dir, f'{pred_dir}.tsv')
            out_df = pd.DataFrame(list(zip(utt_id_list, following_phones, foll_phon_classes, 
                                           preceding_phones, prec_phon_classes,
                                           diffs, accuracy_list)),
               columns =['utt_id', 'phone', 'phone_class', 'prec_phone', 'prec_phone_class', 
                         'diff', 'accuracy'])
            out_df.to_csv(out_df_file, sep='\t', index=False)

    return pd.DataFrame.from_dict(results_dict, orient='index')

In [197]:
tg_dir = 'out/tg'
eval_dirs = ['urum-only2', 'cs-russ-phones3', 'cs-urum-phones3',
            'all-russ-phones3', 'all-urum-phones3',
            'engmfa-urum-only-orig', 'engmfa-cs-russ-phones-orig', 'engmfa-cs-urum-phones-orig',
            'engmfa-all-russ-phones-orig', 'engmfa-all-urum-phones-orig',
            'russmfa-urum-only-orig', 'russmfa-cs-russ-phones-orig', 'russmfa-cs-urum-phones-orig',
            'russmfa-all-urum-phones-orig', 'russmfa-all-russ-phones-orig',
            'urum-only-broad', 'cs-urum-phones-broad', 'all-urum-phones-broad']
out_df = batch_eval_tg(tg_dir, eval_dirs, False, write_df_dir='out/results/test-analysis')
out_df

,# pred utts,# phones,MEAN,MEDIAN,"MIN, MAX",PRECISION,ACCURACY
urum-only2,251,6629,0.036,0.013,"(0.0, 3.31)",62.106,80.615
cs-russ-phones3,255,6671,0.046,0.017,"(0.0, 3.354)",53.860,74.636
cs-urum-phones3,255,6671,0.038,0.015,"(0.0, 2.283)",57.218,77.185
all-russ-phones3,255,6671,0.028,0.010,"(0.0, 4.033)",69.960,85.205
all-urum-phones3,255,6671,0.027,0.010,"(0.0, 2.843)",69.990,85.085
engmfa-urum-only-orig,255,6671,0.049,0.010,"(0.0, 5.44)",69.810,83.766
engmfa-cs-russ-phones-orig,255,6671,0.057,0.010,"(0.0, 5.44)",69.195,83.121
engmfa-cs-urum-phones-orig,255,6671,0.057,0.010,"(0.0, 5.44)",69.255,83.136
engmfa-all-russ-phones-orig,255,6671,0.053,0.010,"(0.0, 5.2)",69.960,83.511
engmfa-all-urum-phones-orig,255,6671,0.053,0.010,"(0.0, 5.44)",70.019,83.601


In [159]:
tg_dir = 'out/tg'
eval_dirs = ['urum-only2', 'cs-russ-phones3', 'cs-urum-phones3',
            'all-russ-phones3', 'all-urum-phones3',
            'engmfa-urum-only', 'engmfa-cs-russ-phones', 'engmfa-cs-urum-phones',
            'engmfa-all-russ-phones', 'engmfa-all-urum-phones',
            'russmfa-urum-only', 'russmfa-cs-russ-phones', 'russmfa-cs-urum-phones',
            'russmfa-all-urum-phones', 'russmfa-all-russ-phones',
            'urum-only-broad', 'cs-urum-phones-broad', 'all-urum-phones-broad']
out_df = batch_eval_tg(tg_dir, eval_dirs, False, write_df_dir='out/results')
out_df

,# pred utts,# phones,MEAN,MEDIAN,"MIN, MAX",PRECISION,ACCURACY
urum-only2,251,6631,0.035,0.013,"(0, 3)",63.279,80.621
cs-russ-phones3,255,6673,0.046,0.017,"(0, 3)",54.668,74.674
cs-urum-phones3,255,6673,0.038,0.016,"(0, 2)",58.160,77.192
all-russ-phones3,255,6673,0.031,0.010,"(0, 4)",70.883,85.089
all-urum-phones3,255,6673,0.027,0.010,"(0, 3)",70.973,85.044
engmfa-urum-only,255,6673,0.044,0.010,"(0, 5)",70.568,84.025
engmfa-cs-russ-phones,255,6673,0.057,0.010,"(0, 5)",69.999,83.111
engmfa-cs-urum-phones,255,6673,0.058,0.010,"(0, 5)",69.999,83.051
engmfa-all-russ-phones,255,6673,0.053,0.010,"(0, 5)",70.643,83.531
engmfa-all-urum-phones,255,6673,0.046,0.010,"(0, 5)",70.913,83.830


In [136]:
eval_tg('out/tg/urum-only2', False, show_nat_class=True)

bracket_word_count: 61
num pred utts: 251
pred file not in gold: 0002_DoReCo_doreco_urum1249_UUM-TXT-CL-00000-B03
pred file not in gold: 0004_DoReCo_doreco_urum1249_UUM-TXT-CL-00000-B03
pred file not in gold: 0006_DoReCo_doreco_urum1249_UUM-TXT-CL-00000-B03
Boundary Onset statistics:
# phones: 6631
MEAN: 0.035
MEDIAN: 0.013000000000000012
MIN, MAX: 0.0, 3.3100000000000005
WINDOW 0.02s (PRECISION) %): 63.279
ACCURACY (test midpoint within gold interval) %: 80.621

PREC (offset)


** Vowels **
WINDOW (0.02s) %: 58.985	COUNT:2660
MEAN: 0.04
MEDIAN: 0.015000000000000124
MIN, MAX: 0.0, 3.1370000000000005

** Fricatives **
WINDOW (0.02s) %: 72.286	COUNT:866
MEAN: 0.03
MEDIAN: 0.010000000000000009
MIN, MAX: 0.0, 3.234

** Affricates **
WINDOW (0.02s) %: 73.913	COUNT:92
MEAN: 0.02
MEDIAN: 0.007499999999999951
MIN, MAX: 0.0, 0.13700000000000045

** Nasals **
WINDOW (0.02s) %: 67.184	COUNT:515
MEAN: 0.03
MEDIAN: 0.01200000000000001
MIN, MAX: 0.0, 2.216

** Approximants **
WINDOW (0.02s) %: 50.28

#### Urum-only2

* Boundary Onset statistics:
* ** OVERALL **
* MEAN: 0.035
* MEDIAN: 0.01300000000000101
* MIN: 0.0
* MAX: 3.309999999999998
* WINDOW (0.02s): 64.017

#### Urum-only2 (134 urum test utts only)

* Boundary Onset statistics:
* ** OVERALL **
* MEAN: 0.031
* MEDIAN: 0.012620357121510856
* MIN: 0.0
* MAX: 2.282999999999998
* WINDOW (0.02s): 65.708

#### CS-urum-phones-3 (134 urum test utts only)

* Boundary Onset statistics:
* ** OVERALL **
* MEAN: 0.038
* MEDIAN: 0.015999999999999792
* MIN: 0.0
* MAX: 2.283
* WINDOW (0.02s): 58.160

#### All-urum-phones3 (134 urum test utts only)

* Boundary Onset statistics:
* ** OVERALL **
* MEAN: 0.027
* MEDIAN: 0.010000000000000009
* MIN: 0.0
* MAX: 2.843
* WINDOW (0.02s): 70.973

#### All-russ-phones3 (134 urum test utts only)

* Boundary Onset statistics:
* ** OVERALL **
* MEAN: 0.031
* MEDIAN: 0.010000000000000009
* MIN: 0.0
* MAX: 4.0329999999999995
* WINDOW (0.02s): 70.883